# 📚 AI Story Generator with Images

This notebook creates an interactive app that generates stories with corresponding images using Hugging Face's transformers and diffusers libraries.

## Features:
- Text generation using pre-trained language models
- Image generation using Stable Diffusion
- Interactive widgets for user input
- Complete story creation with multiple chapters and images

Let's get started!

## 1. Install and Import Required Libraries

First, let's install and import all the necessary libraries for our story generator app.

In [ ]:
# Install required packages
!pip install transformers diffusers torch torchvision accelerate pillow ipywidgets matplotlib

In [ ]:
# Configure network settings for better download stability
import os
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Set longer timeout for model downloads
os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '300'  # 5 minutes
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Avoid warnings

# Configure retry strategy for network requests
def configure_requests_retry():
    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

print("✅ Network configuration applied for stable downloads")

In [ ]:
# Import required libraries
import torch
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from diffusers import StableDiffusionPipeline
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import warnings
import os
import re

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Load Text Generation Model

Let's load a pre-trained text generation model for creating story content. We'll use GPT-2 which is great for creative writing.

In [ ]:
# Load text generation model
print("Loading text generation model...")

# Use GPT-2 medium for better story quality
model_name = "gpt2-medium"
text_generator = pipeline(
    "text-generation",
    model=model_name,
    tokenizer=model_name,
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

print("✅ Text generation model loaded successfully!")
print(f"Model: {model_name}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## 3. Load Image Generation Model

Now let's load a Stable Diffusion model for generating images based on text descriptions from our stories.

In [ ]:
# Load image generation model
print("Loading image generation model...")

# Use appropriate device and data type
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Try multiple model options with fallbacks
model_options = [
    "runwayml/stable-diffusion-v1-5",
    "CompVis/stable-diffusion-v1-4",
    "stabilityai/stable-diffusion-2-1-base"
]

image_generator = None
model_loaded = False

for model_id in model_options:
    try:
        print(f"Attempting to load: {model_id}")

        # Try loading with resume download and local files only if network fails
        image_generator = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch_dtype,
            use_safetensors=True,
            resume_download=True,
            local_files_only=False,
            use_auth_token=False,
            low_cpu_mem_usage=True
        )

        image_generator = image_generator.to(device)

        # Enable memory efficient attention if using GPU
        if torch.cuda.is_available():
            image_generator.enable_attention_slicing()
            image_generator.enable_model_cpu_offload()

        print("✅ Image generation model loaded successfully!")
        print(f"Model: {model_id}")
        print(f"Device: {device}")
        model_loaded = True
        break

    except Exception as e:
        print(f"❌ Failed to load {model_id}: {str(e)[:100]}...")
        continue

if not model_loaded:
    print("⚠️ Could not load any Stable Diffusion model due to network issues.")
    print("💡 Solutions:")
    print("1. Check your internet connection")
    print("2. Try running the cell again (models resume download)")
    print("3. Use a VPN if you're in a restricted region")
    print("4. Try again later when network is more stable")
    print("📝 The text generation will still work - only images will be affected.")

### 🔄 Troubleshooting Model Download Issues

If you encounter network timeout errors when downloading models:

1. **Try running the cell again** - Downloads will resume from where they left off
2. **Check your internet connection** - Large model files require stable connection
3. **Use alternative model** - The code will try multiple Stable Diffusion versions
4. **Manual download** - You can pre-download models using the code below if needed

In [ ]:
# Manual retry for image model loading (run this if the above cell fails)
# Uncomment and run this cell if you're still having download issues

"""
print("🔄 Manual retry with different settings...")

import time
from huggingface_hub import snapshot_download

# Try downloading the model files manually with retry
model_id = "runwayml/stable-diffusion-v1-5"
max_retries = 3

for attempt in range(max_retries):
    try:
        print(f"Attempt {attempt + 1}/{max_retries}: Downloading {model_id}...")

        # Download model files manually
        snapshot_download(
            repo_id=model_id,
            resume_download=True,
            local_files_only=False,
            ignore_patterns=["*.bin"]  # Use safetensors only
        )

        # Now try loading the model
        image_generator = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch_dtype,
            use_safetensors=True,
            local_files_only=True  # Use downloaded files
        )

        image_generator = image_generator.to(device)

        if torch.cuda.is_available():
            image_generator.enable_attention_slicing()
            image_generator.enable_model_cpu_offload()

        print("✅ Manual download and loading successful!")
        break

    except Exception as e:
        print(f"❌ Attempt {attempt + 1} failed: {str(e)[:100]}...")
        if attempt < max_retries - 1:
            print("⏳ Waiting 30 seconds before retry...")
            time.sleep(30)
        else:
            print("💡 All attempts failed. Story generation will work without images.")
"""

print("💡 Uncomment and run the code above if you need manual retry for model loading")

## 4. Create Story Generation Function

Let's implement a function that generates coherent story text based on a given prompt.

In [ ]:
def generate_story_chapter(prompt, max_length=200, temperature=0.8, num_return_sequences=1):
    """
    Generate a story chapter based on a given prompt.

    Args:
        prompt (str): The starting prompt for the story
        max_length (int): Maximum length of generated text
        temperature (float): Controls randomness (0.1 = conservative, 1.0 = creative)
        num_return_sequences (int): Number of different versions to generate

    Returns:
        str: Generated story text
    """
    try:
        # Generate text
        generated = text_generator(
            prompt,
            max_length=max_length,
            temperature=temperature,
            num_return_sequences=num_return_sequences,
            pad_token_id=text_generator.tokenizer.eos_token_id,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1
        )

        # Extract the generated text
        story_text = generated[0]['generated_text']

        # Clean up the text (remove the original prompt and clean formatting)
        if story_text.startswith(prompt):
            story_text = story_text[len(prompt):].strip()

        return story_text

    except Exception as e:
        return f"Error generating story: {str(e)}"

def extract_scene_descriptions(text):
    """
    Extract visual scene descriptions from story text for image generation.

    Args:
        text (str): Story text

    Returns:
        list: List of scene descriptions suitable for image generation
    """
    # Simple pattern matching to find descriptive sentences
    sentences = text.split('.')
    scene_descriptions = []

    # Look for sentences with visual descriptors
    visual_keywords = ['looked', 'saw', 'appeared', 'stood', 'walked', 'dark', 'bright',
                      'beautiful', 'scary', 'ancient', 'mysterious', 'golden', 'silver',
                      'forest', 'castle', 'mountain', 'ocean', 'sky', 'moon', 'sun']

    for sentence in sentences:
        sentence = sentence.strip()
        if len(sentence) > 20 and any(keyword in sentence.lower() for keyword in visual_keywords):
            # Clean and format for image generation
            clean_sentence = re.sub(r'[^\w\s,.-]', '', sentence)
            if len(clean_sentence) > 10:
                scene_descriptions.append(clean_sentence)

    return scene_descriptions[:2]  # Return max 2 scenes per chapter

# Test the story generation function
test_prompt = "Once upon a time, in a magical forest"
print("Testing story generation...")
test_story = generate_story_chapter(test_prompt, max_length=150)
print(f"✅ Story generation test successful!")
print(f"Sample output: {test_story[:100]}...")

## 5. Create Image Generation Function

Now let's implement a function that generates images based on text descriptions from our stories.

In [ ]:
def generate_story_image(prompt, style="fantasy art, detailed, high quality", negative_prompt="blurry, low quality"):
    """
    Generate an image based on a text description from the story.

    Args:
        prompt (str): Text description to generate image from
        style (str): Art style specification
        negative_prompt (str): What to avoid in the image

    Returns:
        PIL.Image: Generated image
    """
    # Check if image generator is available
    if 'image_generator' not in globals() or image_generator is None:
        print("⚠️ Image generator not available. Creating placeholder image...")
        return create_placeholder_image(prompt)

    try:
        # Enhance the prompt with style information
        enhanced_prompt = f"{prompt}, {style}"

        # Generate image with timeout handling
        with torch.no_grad():
            image = image_generator(
                enhanced_prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=20,  # Reduced for faster generation
                guidance_scale=7.5,
                height=512,
                width=512
            ).images[0]

        return image

    except Exception as e:
        print(f"Error generating image: {str(e)[:100]}...")
        # Return a placeholder image with the text description
        return create_placeholder_image(prompt)

def create_placeholder_image(text, size=(512, 512)):
    """
    Create a simple placeholder image with text when image generation fails.

    Args:
        text (str): Text to display on placeholder
        size (tuple): Image size

    Returns:
        PIL.Image: Placeholder image
    """
    from PIL import ImageDraw, ImageFont

    # Create a light blue background
    img = Image.new('RGB', size, color='lightblue')
    draw = ImageDraw.Draw(img)

    # Try to use a default font, fallback to basic if not available
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()

    # Add placeholder text
    placeholder_text = "🖼️ IMAGE PLACEHOLDER\n\n" + text[:60] + "..."

    # Calculate text position (centered)
    bbox = draw.textbbox((0, 0), placeholder_text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]

    x = (size[0] - text_width) // 2
    y = (size[1] - text_height) // 2

    # Draw text with shadow effect
    draw.text((x+2, y+2), placeholder_text, fill='gray', font=font, align='center')
    draw.text((x, y), placeholder_text, fill='black', font=font, align='center')

    return img

def display_image_with_caption(image, caption, max_width=400):
    """
    Display an image with a caption in the notebook.

    Args:
        image (PIL.Image): Image to display
        caption (str): Caption text
        max_width (int): Maximum display width
    """
    plt.figure(figsize=(max_width/100, max_width/100))
    plt.imshow(image)
    plt.axis('off')
    plt.title(caption, fontsize=12, wrap=True)
    plt.tight_layout()
    plt.show()

print("✅ Image generation functions created successfully!")

In [ ]:
# Test image generation to verify it's working
print("🧪 Testing image generation...")

# Test if the image_generator variable exists and is properly loaded
if 'image_generator' in globals() and image_generator is not None:
    print("✅ Image generator is loaded and available")
    print(f"Device: {image_generator.device}")
    
    # Try generating a simple test image
    try:
        print("🎨 Generating test image...")
        test_image = generate_story_image("a magical forest with glowing trees", style="fantasy art, detailed")
        
        if test_image:
            print("✅ Image generation successful!")
            display_image_with_caption(test_image, "Test Image: Magical Forest")
        else:
            print("❌ Image generation returned None")
            
    except Exception as e:
        print(f"❌ Error during image generation: {str(e)}")
        print("💡 This might be due to memory issues or model loading problems")
        
else:
    print("❌ Image generator is not loaded")
    print("💡 Please run the image model loading cell first")

## 6. Build Interactive Story App

Let's create an interactive widget-based app that allows users to input prompts and generate stories with images.

In [ ]:
class StoryGeneratorApp:
    def __init__(self):
        self.current_story = ""
        self.story_images = []
        self.chapter_count = 0

        # Create widgets
        self.story_prompt = widgets.Textarea(
            value="Once upon a time, in a magical kingdom",
            placeholder="Enter your story prompt here...",
            description="Story Prompt:",
            layout=widgets.Layout(width='80%', height='100px'),
            style={'description_width': 'initial'}
        )

        self.chapter_length = widgets.IntSlider(
            value=150,
            min=50,
            max=300,
            step=25,
            description="Chapter Length:",
            style={'description_width': 'initial'}
        )

        self.creativity = widgets.FloatSlider(
            value=0.8,
            min=0.1,
            max=1.5,
            step=0.1,
            description="Creativity:",
            style={'description_width': 'initial'}
        )

        self.art_style = widgets.Dropdown(
            options=[
                'fantasy art, detailed, high quality',
                'realistic, photographic, detailed',
                'anime style, colorful, detailed',
                'watercolor painting, artistic',
                'digital art, vibrant, detailed',
                'oil painting, classical, detailed'
            ],
            value='fantasy art, detailed, high quality',
            description="Art Style:",
            style={'description_width': 'initial'}
        )

        self.generate_button = widgets.Button(
            description="📖 Generate Chapter",
            button_style='primary',
            layout=widgets.Layout(width='200px')
        )

        self.continue_button = widgets.Button(
            description="➡️ Continue Story",
            button_style='info',
            layout=widgets.Layout(width='200px')
        )

        self.reset_button = widgets.Button(
            description="🔄 New Story",
            button_style='warning',
            layout=widgets.Layout(width='200px')
        )

        self.output_area = widgets.Output()

        # Bind button events
        self.generate_button.on_click(self.generate_chapter)
        self.continue_button.on_click(self.continue_story)
        self.reset_button.on_click(self.reset_story)

    def generate_chapter(self, button):
        """Generate a new chapter and corresponding images."""
        with self.output_area:
            clear_output(wait=True)

            prompt = self.story_prompt.value
            if not prompt.strip():
                print("❌ Please enter a story prompt!")
                return

            print("🔄 Generating story chapter...")

            # Generate story text
            chapter_text = generate_story_chapter(
                prompt,
                max_length=self.chapter_length.value,
                temperature=self.creativity.value
            )

            self.chapter_count += 1
            self.current_story += f"\\n\\n**Chapter {self.chapter_count}**\\n{chapter_text}"

            # Extract scene descriptions for images
            scene_descriptions = extract_scene_descriptions(chapter_text)

            # Display the chapter
            print(f"📖 **Chapter {self.chapter_count}**")
            print("-" * 50)
            print(chapter_text)
            print("\\n")

            # Generate and display images
            if scene_descriptions:
                print("🎨 Generating images...")
                for i, scene in enumerate(scene_descriptions):
                    try:
                        image = generate_story_image(scene, style=self.art_style.value)
                        self.story_images.append((f"Chapter {self.chapter_count} - Scene {i+1}", image))
                        display_image_with_caption(image, f"Scene {i+1}: {scene[:60]}...")
                    except Exception as e:
                        print(f"❌ Error generating image for scene {i+1}: {e}")

            print("\\n✅ Chapter generated successfully!")

            # Update prompt for continuation
            last_sentence = chapter_text.split('.')[-2] + '.' if '.' in chapter_text else chapter_text[-50:]
            self.story_prompt.value = last_sentence

    def continue_story(self, button):
        """Continue the current story."""
        if not self.current_story:
            with self.output_area:
                print("❌ No story to continue! Generate a chapter first.")
            return

        self.generate_chapter(button)

    def reset_story(self, button):
        """Reset the story and start fresh."""
        self.current_story = ""
        self.story_images = []
        self.chapter_count = 0
        self.story_prompt.value = "Once upon a time, in a magical kingdom"

        with self.output_area:
            clear_output()
            print("🔄 Story reset! Ready for a new adventure.")

    def display_app(self):
        """Display the complete story generator app."""
        # Create layout
        controls = widgets.VBox([
            widgets.HTML("<h3>🎯 Story Settings</h3>"),
            self.story_prompt,
            widgets.HBox([self.chapter_length, self.creativity]),
            self.art_style,
            widgets.HTML("<h3>🎮 Controls</h3>"),
            widgets.HBox([self.generate_button, self.continue_button, self.reset_button]),
            widgets.HTML("<hr>"),
            widgets.HTML("<h3>📚 Generated Story</h3>")
        ])

        # Display the app
        display(controls)
        display(self.output_area)

# Create and display the app
print("🚀 Creating Story Generator App...")
story_app = StoryGeneratorApp()
print("✅ App created successfully! Use the interface below to generate your story.")

## 7. Generate Complete Story with Images

Let's create a function that generates a complete story with multiple chapters and corresponding images automatically.

In [ ]:
def generate_complete_story(initial_prompt, num_chapters=3, chapter_length=150, temperature=0.8, art_style="fantasy art, detailed, high quality"):
    """
    Generate a complete story with multiple chapters and images.

    Args:
        initial_prompt (str): Starting prompt for the story
        num_chapters (int): Number of chapters to generate
        chapter_length (int): Length of each chapter
        temperature (float): Creativity level
        art_style (str): Art style for images

    Returns:
        dict: Complete story with text and images
    """
    story_data = {
        'chapters': [],
        'images': [],
        'full_text': ''
    }

    current_prompt = initial_prompt

    print(f"📚 Generating a {num_chapters}-chapter story...")
    print("=" * 60)

    for chapter_num in range(1, num_chapters + 1):
        print(f"\\n🔄 Generating Chapter {chapter_num}...")

        # Generate chapter text
        chapter_text = generate_story_chapter(
            current_prompt,
            max_length=chapter_length,
            temperature=temperature
        )

        # Store chapter
        story_data['chapters'].append({
            'number': chapter_num,
            'text': chapter_text,
            'prompt': current_prompt
        })

        story_data['full_text'] += f"\\n\\n**Chapter {chapter_num}**\\n{chapter_text}"

        # Display chapter
        print(f"\\n📖 **Chapter {chapter_num}**")
        print("-" * 40)
        print(chapter_text)

        # Extract and generate images
        scene_descriptions = extract_scene_descriptions(chapter_text)
        chapter_images = []

        if scene_descriptions:
            print(f"\\n🎨 Generating {len(scene_descriptions)} image(s) for Chapter {chapter_num}...")

            for i, scene in enumerate(scene_descriptions):
                try:
                    image = generate_story_image(scene, style=art_style)
                    image_info = {
                        'chapter': chapter_num,
                        'scene': i + 1,
                        'description': scene,
                        'image': image
                    }
                    chapter_images.append(image_info)
                    story_data['images'].append(image_info)

                    # Display image
                    display_image_with_caption(
                        image,
                        f"Chapter {chapter_num}, Scene {i+1}: {scene[:50]}..."
                    )

                except Exception as e:
                    print(f"❌ Error generating image for scene {i+1}: {e}")

        # Prepare prompt for next chapter (use last sentence + transition)
        if chapter_num < num_chapters:
            last_sentences = chapter_text.split('.')[-3:-1]  # Get last 2 sentences
            transition_prompt = '. '.join(last_sentences) + '. Meanwhile,'
            current_prompt = transition_prompt

        print(f"✅ Chapter {chapter_num} completed!")

    print("\\n" + "=" * 60)
    print(f"🎉 Complete story generated successfully!")
    print(f"📊 Total chapters: {len(story_data['chapters'])}")
    print(f"🖼️ Total images: {len(story_data['images'])}")

    return story_data

# Example usage function
def create_sample_story():
    """Create a sample story to demonstrate the complete generation."""
    sample_prompts = [
        "In a mystical forest where ancient trees whispered secrets",
        "On a distant planet where two moons cast silver light",
        "In a steampunk city where gears and magic coexisted",
        "Deep in the ocean where mermaids guarded ancient treasures",
        "In a floating castle above the clouds where wizards studied"
    ]

    print("🎲 Choose a sample prompt or create your own:")
    for i, prompt in enumerate(sample_prompts, 1):
        print(f"{i}. {prompt}")

    return sample_prompts

sample_prompts = create_sample_story()
print("\\n✅ Complete story generation functions ready!")

## 8. Display Results

Let's create the final interface and launch our story generator app!

In [ ]:
# Launch the Interactive Story Generator App
print("🚀 Launching AI Story Generator with Images!")
print("=" * 60)

# Display app interface
story_app.display_app()

print("\\n" + "💡" * 20 + " USAGE TIPS " + "💡" * 20)
print("""
🎯 How to use the Story Generator:

1. **Enter a Story Prompt**: Write your story beginning in the text area
2. **Adjust Settings**:
   - Chapter Length: Controls how long each chapter will be
   - Creativity: Higher values = more creative/unpredictable stories
   - Art Style: Choose the visual style for generated images

3. **Generate Chapter**: Click to create the first chapter with images
4. **Continue Story**: Add more chapters to your story
5. **New Story**: Reset and start a completely new story

🎨 **Art Styles Available**:
- Fantasy Art: Magical, mystical scenes
- Realistic: Photographic quality images
- Anime Style: Colorful, animated look
- Watercolor: Artistic, painted effect
- Digital Art: Modern, vibrant style
- Oil Painting: Classical, detailed artwork

⚡ **Performance Tips**:
- Image generation may take 10-30 seconds per image
- Use shorter chapters for faster generation
- Lower creativity settings generate more coherent text
- GPU acceleration will significantly speed up image generation

🎭 **Story Ideas to Try**:
- "In a cyberpunk city where AI and humans coexist..."
- "On a space station orbiting a dying star..."
- "In a magical school hidden in the mountains..."
- "During the last day on Earth before evacuation..."
- "In a world where books come to life..."
""")

## 🎬 Example: Generate a Complete Story

Want to generate a complete multi-chapter story automatically? Run the cell below with your chosen prompt!

In [ ]:
# Example: Generate a complete story with images
# Uncomment and modify the line below to generate an automatic story

# Choose your story prompt
story_prompt = "In a hidden valley where dragons and humans lived in harmony"

# Uncomment the line below to generate a complete 3-chapter story
# complete_story = generate_complete_story(
#     initial_prompt=story_prompt,
#     num_chapters=3,
#     chapter_length=120,
#     temperature=0.8,
#     art_style="fantasy art, detailed, high quality"
# )

print("📝 To generate a complete story automatically:")
print("1. Choose or modify the story_prompt above")
print("2. Uncomment the generate_complete_story() function call")
print("3. Run this cell")
print("\\n⚠️ Note: This will take several minutes to complete!")

# Alternative: Use one of the sample prompts
print("\\n🎲 Or choose from these sample prompts:")
for i, prompt in enumerate(sample_prompts, 1):
    print(f"{i}. {prompt}")

## 🔧 Troubleshooting & Information

### System Requirements
- **Memory**: At least 8GB RAM (16GB recommended for image generation)
- **GPU**: CUDA-compatible GPU recommended for faster image generation
- **Storage**: ~5GB free space for model downloads

### Common Issues & Solutions

**1. Out of Memory Errors**
- Reduce chapter length or image resolution
- Enable model CPU offloading (already implemented)
- Use lower precision (float16 vs float32)

**2. Slow Generation**
- First run downloads models (~4GB), subsequent runs are faster
- GPU acceleration significantly improves speed
- Reduce number of inference steps for images (already optimized)

**3. Poor Quality Output**
- Adjust creativity/temperature settings
- Try different art styles
- Modify prompts to be more descriptive

### Model Information
- **Text Model**: GPT-2 Medium (~350M parameters)
- **Image Model**: Stable Diffusion v1.5 (~860M parameters)
- **Total Download**: ~4-5GB on first run

### Customization Options
- Change text generation models (GPT-2, GPT-Neo, etc.)
- Modify image generation models (different Stable Diffusion versions)
- Add custom art styles and prompts
- Implement story structure templates

### Legal & Ethical Considerations
- Generated content is for personal/educational use
- Be mindful of content guidelines when sharing
- Models may reflect biases from training data
- Always review generated content before sharing

---

**🎉 Enjoy creating amazing stories with AI! 🎉**